# Generators

In Python there are often many different ways to accomplish a single task.  So far you are familiar with many of Pythons primitive data types - specifically for this discussion we will focus on data types that can be iterated over.  

A list is a perfect example of a type that can be iterated over.  When Python iterated through the list there are a few major things to note about how it does it:

 - The list is fully loaded into memory
 - Each individual item in the list is exposed to the scope as the variable name you provided.
 
List iteration for review:

# Named Tuple

Named Tuples (from here on referred to by their syntactically correct name `namedtuple`) are possibly the quickest way to convert a primitive data structure into a Python object-like instance.  It generates a hefty amount of boilerplate code for you and all you really pass in is a Name, Keys, and Values. A `namedtuple` can be especially useful if you're working with something like a JSON Api that will pass you a flat string of data that you need to convert into something useful.  Lets see an example: 
